# Data (+Stats) for Paper

In [1]:
#import libraries
import pandas as pd
import numpy as np
import netCDF4
import xarray as xr

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import scipy
import scipy.stats
from scipy import signal
from scipy.signal import butter, lfilter, sosfilt

import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature

import seaborn as sns

In [2]:
#set up for wrf and era
variables_wrf = ['T2', 'SWDNT']
variables_era = ['mx2t']
variables_era_gph = ['z']
variables_wrf_gph = ['gph', 'temp']
temperature_variables_era = ["t2m"]

### Grab Data

In [3]:
#time indices
indices = np.arange(57,153+1)

#ERA for records
with xr.open_dataset("/scratch/rwhite/ladmasu/insol/era5_t2m_hourly_1940-2021_PNW_JJA.nc") as era_reg:
    era_dailymax = era_reg.resample(time = "1D").max()
    #find daily maximum and find the 2021 event in era5
    era_daily_max_PNW = era_dailymax.sel(latitude = slice(40,65), longitude = slice(-140,-100), 
                       time = slice("1940-01-01T00:00:00.000000000", "2021-06-01T00:00:00.000000000")).t2m.max(dim = "time")
    era_test_hw = era_dailymax.sel(latitude = slice(40,65), longitude = slice(-140, -100), 
                       time = slice("2021-06-26T00:00:00.000000000", "2021-06-30T00:00:00.000000000")).t2m.max(dim = "time")

In [4]:
##### MAIN WRF RUNS (REGRIDDED FOR RECORDS) #####
with xr.open_dataset("/scratch/rwhite/ladmasu/insol/new_dat/BIAS_FIXED/rrtmg_radiation/june_d02_timemerge_rrtmg_regridera2d.nc") as june_2d:
    weights = np.cos(np.deg2rad(june_2d.sel(latitude = slice(65,40)).latitude))
    weighted_weights = weights/np.mean(weights)
    
    #grab june data for maps and timeseries
    june2d_bc = june_2d.sel(latitude = slice(65,40), longitude = slice(-140, -100), XTIME = june_2d.XTIME[indices])
    
    june2d_bcw = june2d_bc.groupby('latitude')*weighted_weights
    junedata_weighted = june2d_bcw.mean(dim = ['latitude', 'longitude']).T2
    
with xr.open_dataset("/scratch/rwhite/ladmasu/insol/new_dat/BIAS_FIXED/rrtmg_radiation/jul_d02_timemerge_rrtmg_regridera2d.nc") as july_2d:
    #grab july data for maps and timeseries
    july2d_bc = july_2d.sel(latitude = slice(65,40), longitude = slice(-140, -100), XTIME = july_2d.XTIME[indices])
    
    july2d_bcw = july2d_bc.groupby('latitude')*weighted_weights
    julydata_weighted = july2d_bcw.mean(dim = ['latitude', 'longitude']).T2

with xr.open_dataset("/scratch/rwhite/ladmasu/insol/new_dat/BIAS_FIXED/rrtmg_radiation/aug_d02_timemerge_rrtmg_regridera2d.nc") as aug_2d:
    #grab june data for maps and timeseries
    aug2d_bc = aug_2d.sel(latitude = slice(65,40), longitude = slice(-140, -100), XTIME = aug_2d.XTIME[indices])
    aug2d_bcw = aug2d_bc.groupby('latitude')*weighted_weights
    augdata_weighted = aug2d_bcw.mean(dim = ['latitude', 'longitude']).T2

In [24]:
#DATA FOR SENSITIVITY STUDIES
#Initialisation Date
with xr.open_dataset('/scratch/rwhite/ladmasu/insol/sensitivity/june_minus2_d02_timemerge_regridera2d.nc') as june_minus2:
    june_minus2 = june_minus2.sel(latitude = slice(65,40), longitude = slice(-140, -100), XTIME = june_minus2.XTIME[indices+48])
    june_minus2w = june_minus2.groupby("latitude")*weighted_weights
    june_minus2T = june_minus2w.mean(dim = ['latitude', 'longitude']).T2
    june_minus2T_map = june_minus2w.mean(dim = "XTIME").T2
    
with xr.open_dataset('/scratch/rwhite/ladmasu/insol/sensitivity/aug_minus2_d02_timemerge_regridera2d.nc') as aug_minus2:
    aug_minus2 = aug_minus2.sel(latitude = slice(65,40), longitude = slice(-140, -100), XTIME = aug_minus2.XTIME[indices+48])
    aug_minus2w = aug_minus2.groupby("latitude")*weighted_weights
    aug_minus2T = aug_minus2w.mean(dim = ['latitude', 'longitude']).T2
    aug_minus2T_map = aug_minus2w.mean(dim = "XTIME").T2

#CAM    
with xr.open_dataset("/scratch/rwhite/ladmasu/insol/new_dat/BIAS_FIXED/cam_radiation/june_d02_timemerge_cam_regridera.nc") as june_cam:
    june_cam = june_cam.sel(latitude = slice(65,40), longitude = slice(-140, -100), XTIME = june_cam.XTIME[indices])
    june_camw = june_cam.groupby("latitude")*weighted_weights
    june_camT = june_camw.mean(dim = ['latitude', 'longitude']).T2
    june_camT_map = june_camw.mean(dim = "XTIME").T2
    
with xr.open_dataset("/scratch/rwhite/ladmasu/insol/new_dat/BIAS_FIXED/cam_radiation/aug_d02_timemerge_cam_regridera.nc") as aug_cam:
    aug_cam = aug_cam.sel(latitude = slice(65,40), longitude = slice(-140, -100), XTIME = aug_cam.XTIME[indices])
    aug_camw = aug_cam.groupby("latitude")*weighted_weights
    aug_camT = aug_camw.mean(dim = ['latitude', 'longitude']).T2
    aug_camT_map = aug_camw.mean(dim = "XTIME").T2
    
#NOAH LSM
with xr.open_dataset("/scratch/rwhite/ladmasu/insol/sensitivity/june_oldNOAHLSM-rrtmg_d02_timemerge_regridera2d.nc") as june_lsm:
    june_lsm = june_lsm.sel(latitude = slice(65,40), longitude = slice(-140, -100), XTIME = june_lsm.XTIME[indices])
    june_lsmw = june_lsm.groupby("latitude")*weighted_weights
    june_lsmT = june_lsmw.mean(dim = ['latitude', 'longitude']).T2
    june_lsmT_map = june_lsmw.mean(dim = "XTIME").T2
    
with xr.open_dataset("/scratch/rwhite/ladmasu/insol/sensitivity/aug_oldNOAHLSM-rrtmg_d02_timemerge_regridera2d.nc") as aug_lsm:
    aug_lsm = aug_lsm.sel(latitude = slice(65,40), longitude = slice(-140, -100), XTIME = aug_lsm.XTIME[indices])
    aug_lsmw = aug_lsm.groupby("latitude")*weighted_weights
    aug_lsmT = aug_lsmw.mean(dim = ['latitude', 'longitude']).T2
    aug_lsmT_map = aug_lsmw.mean(dim = "XTIME").T2

# #3D DATA
june = xr.open_dataset('/scratch/rwhite/ladmasu/insol/new_dat/BIAS_FIXED/rrtmg_radiation/june_d02_reproj_rrtmg_regrid_3d.nc')
july = xr.open_dataset('/scratch/rwhite/ladmasu/insol/new_dat/BIAS_FIXED/rrtmg_radiation/july_d02_reproj_rrtmg_regrid_3d.nc')
aug = xr.open_dataset('/scratch/rwhite/ladmasu/insol/new_dat/BIAS_FIXED/rrtmg_radiation/aug_d02_reproj_rrtmg_regrid_3d.nc')

sens_3d = xr.open_dataset('/scratch/rwhite/ladmasu/insol/new_dat/BIAS_FIXED/cam_radiation/june_d02_reproj_cam_regrid_3d.nc')

#mask for records
mask_file = xr.open_dataset('/scratch/rwhite/ladmasu/insol/better_mask_regrid.nc')
mask = (mask_file/mask_file).sel(latitude = slice(40,65), longitude = slice(-140, -100)).mean(dim = "time").t2m

In [25]:
#functions
def records_area(records_broken, model_june, model, threshold, area_path, land_ocean_mask):
    area_dat = xr.open_dataset(area_path)
    area = area_dat.cell_area.sel(latitude = slice(65,40), longitude = slice(-140,-100)) #find PNW
    mask = land_ocean_mask
    #find area
    records = records_broken - (model_june-model)
    records_land = mask.values*records.copy()
    #make normalised map of places (not ocean) where records were broken
    plot = records_land/records_land
    #calcualte area where records were broken
    area_rev = area.reindex(latitude=list(reversed(area.latitude)))
    area_above_threshold = area_rev.where(records_land>threshold,0).sum()/1000**2 #in km^2
    return(area_above_threshold, plot)

def mean_temperature(data):
    """find latitude weighted mean average temperature of data over the heatwave box"""
    weights = np.cos(np.deg2rad(data.sel(latitude = slice(52,45)).latitude))/np.mean(np.cos(np.deg2rad(data.sel(latitude = slice(52,45)).latitude)))
    dataw = data.sel(latitude = slice(52,45), longitude = slice(-123,-119)).T2.groupby("latitude")*weights
    data_mean = dataw.mean(dim = ["longitude", "latitude", "XTIME"]) - 273.15
    return data_mean

def max_temperature(data):
    """finds the latitude weighted mean of maximum temperature map over heatwave box"""
    weights = np.cos(np.deg2rad(data.sel(latitude = slice(52,45)).latitude))/np.mean(np.cos(np.deg2rad(data.sel(latitude = slice(52,45)).latitude)))
    dataw = data.sel(latitude = slice(52,45), longitude = slice(-123,-119)).T2.groupby("latitude")*weights
    data_max = dataw.max(dim = "XTIME")
    data_mean = data_max.mean(dim = ["longitude", "latitude"]) - 273.15
    return data_mean

def mean_radiation(data):
    """finds the latitude weighted mean average radiation over the heatwave box"""
    weights = np.cos(np.deg2rad(data.sel(latitude = slice(52,45)).latitude))/np.mean(np.cos(np.deg2rad(data.sel(latitude = slice(52,45)).latitude)))
    dataw = data.sel(latitude = slice(52,45), longitude = slice(-123,-119)).SWDNB.groupby("latitude")*weights
    data_mean = dataw.mean(dim = ["longitude", "latitude", "XTIME"])
    return data_mean

### Temperatures (Max and Mean)

In [42]:
## Main Runs
#mean temperature
june_meanT = mean_temperature(june_2d)
july_meanT = mean_temperature(july_2d)
aug_meanT = mean_temperature(aug_2d)
print(f'MEAN TEMP: Control: {round(float(june_meanT.values),4)}, +30day: {round(float(july_meanT.values),4)}, +60day: {round(float(aug_meanT.values),4)}')

#maximum temperature
june_maxT = max_temperature(june_2d)
july_maxT = max_temperature(july_2d)
aug_maxT = max_temperature(aug_2d)
print(f'MAX TEMP: Control: {round(float(june_maxT.values),4)}, +30day: {round(float(july_maxT.values),4)}, +60day: {round(float(aug_maxT.values),4)}')

## Sensitivity Runs
cam_meanT_june = mean_temperature(june_cam)
noah_meanT_june = mean_temperature(june_lsm)
minus2_meanT_june = mean_temperature(june_minus2)
print(f'MEAN TEMP FOR CONTROL: CAM: {round(float(cam_meanT_june.values),4)}, NOAH MP: {round(float(noah_meanT_june.values),4)}, -2day: {round(float(minus2_meanT_june.values),4)}')

cam_meanT_aug = mean_temperature(aug_cam)
noah_meanT_aug = mean_temperature(aug_lsm)
minus2_meanT_aug = mean_temperature(aug_minus2)
print(f'MEAN TEMP 60day: CAM: {round(float(cam_meanT_aug.values),4)}, NOAH MP: {round(float(noah_meanT_aug.values),4)}, -2day JUNE: {round(float(minus2_meanT_aug.values),4)}')

MEAN TEMP: Control: 27.2634, +30day: 25.9555, +60day: 22.8239
MAX TEMP: Control: 39.7809, +30day: 38.2706, +60day: 34.9148
MEAN TEMP FOR CONTROL: CAM: 29.4019, NOAH MP: 27.1546, -2day: 26.0753
MEAN TEMP 60day: CAM: 24.8874, NOAH MP: 22.8166, -2day JUNE: 22.2243


### Radiation

In [27]:
june_meanRad = mean_radiation(june_2d)
july_meanRad = mean_radiation(july_2d)
aug_meanRad = mean_radiation(aug_2d)
print(f'MEAN RADIATION: Control: {round(float(june_meanRad.values),4)}, +30day: {round(float(july_meanRad.values),4)}, +60day: {round(float(aug_meanRad.values),4)}')

MEAN RADIATION: Control: 364.5845, +30day: 324.1532, +60day: 269.5092


In [18]:
## find maximum radiation differences
june_rad = june2d_bc.sel(latitude = slice(52,45), longitude = slice(-123,-119)).SWDNB.groupby('latitude')*weighted_weights
july_rad = july2d_bc.sel(latitude = slice(52,45), longitude = slice(-123,-119)).SWDNB.groupby('latitude')*weighted_weights
aug_rad = aug2d_bc.sel(latitude = slice(52,45), longitude = slice(-123,-119)).SWDNB.groupby('latitude')*weighted_weights

july_raddiff = (june_rad - july_rad).mean(dim = ['latitude', 'longitude'])
aug_raddiff = (june_rad - aug_rad).mean(dim = ['latitude', 'longitude'])

# julyrad_diffmax = july_raddiff.max()
# augrad_diffmax = aug_raddfiff.max()

## Records

In [28]:
##Main Runs
records_broken = (era_test_hw - era_daily_max_PNW)

#find maximum temperature at each gridpoint
june_max = june2d_bc.max(dim = 'XTIME').T2
july_max = july2d_bc.max(dim = 'XTIME').T2
aug_max = aug2d_bc.max(dim = 'XTIME').T2
#(any record exceedance)
june_area0, june_mask2 = records_area(records_broken, june_max, june_max, 0,'/scratch/rwhite/ladmasu/insol/area_gridera.nc', mask)
july_area0 ,july_mask2= records_area(records_broken, june_max, july_max, 0,'/scratch/rwhite/ladmasu/insol/area_gridera.nc', mask)
aug_area0, aug_mask2 = records_area(records_broken, june_max, aug_max, 0,'/scratch/rwhite/ladmasu/insol/area_gridera.nc', mask)
print(f'TOTAL RECORD EXCEEDANCE: Control: {round(float(june_area0.values),4)}, +30day: {round(float(july_area0.values),4)}, +60day: {round(float(aug_area0.values),4)}')
#3 degree record exceedance
#(any record exceedance)
june_area3, june_mask2 = records_area(records_broken, june_max, june_max, 3,'/scratch/rwhite/ladmasu/insol/area_gridera.nc', mask)
july_area3 ,july_mask2= records_area(records_broken, june_max, july_max, 3,'/scratch/rwhite/ladmasu/insol/area_gridera.nc', mask)
aug_area3, aug_mask2 = records_area(records_broken, june_max, aug_max, 3,'/scratch/rwhite/ladmasu/insol/area_gridera.nc', mask)

print(f'TOTAL RECORD EXCEEDANCE: Control: {round(float(june_area3.values),4)}, +30day: {round(float(july_area3.values),4)}, +60day: {round(float(aug_area3.values),4)}')

TOTAL RECORD EXCEEDANCE: Control: 1792784.7124, +30day: 1349456.1985, +60day: 340980.3933
TOTAL RECORD EXCEEDANCE: Control: 595011.8474, +30day: 322113.5234, +60day: 28962.5875


In [39]:
##sensitivity studies
june_minus2_max = june_minus2.max(dim = "XTIME").T2
june_cam_max = june_cam.max(dim = "XTIME").T2
june_lsm_max = june_lsm.max(dim = "XTIME").T2

aug_minus2_max = aug_minus2.max(dim = "XTIME").T2
aug_cam_max = aug_cam.max(dim = "XTIME").T2
aug_lsm_max = aug_lsm.max(dim = "XTIME").T2

#(any record exceedance)
minus2_area0, a = records_area(records_broken, june_minus2_max, aug_minus2_max, 0,'/scratch/rwhite/ladmasu/insol/area_gridera.nc', mask)
cam_area0 ,b= records_area(records_broken, june_cam_max, aug_cam_max, 0,'/scratch/rwhite/ladmasu/insol/area_gridera.nc', mask)
lsm_area0, c = records_area(records_broken, june_lsm_max, aug_lsm_max, 0,'/scratch/rwhite/ladmasu/insol/area_gridera.nc', mask)

print(f'TOTAL RECORD EXCEEDANCE: Minus 2: {round(float(minus2_area0.values),4)}, CAM: {round(float(cam_area0.values),4)}, NOAH: {round(float(lsm_area0.values),4)}')

TOTAL RECORD EXCEEDANCE: Minus 2: 308970.833, CAM: 332536.7748, NOAH: 271464.9277


### GPH comparison (CTRL and CAM Rad)

In [12]:
rrtmg_gph = june.sel(lat = slice(45,52), lon = slice(-123,-119), Time = june.Time[indices], level = 500).gph
cam_gph = sens_3d.sel(lat = slice(45,52), lon = slice(-123,-119), Time = june.Time[indices], level = 500).gph
diff_sensgph = rrtmg_gph.mean(dim = "Time") - cam_gph.mean(dim = "Time")

july_gph = july.sel(lat = slice(45,52), lon = slice(-123,-119), Time = july.Time[indices], level = 500).gph
diff_months = rrtmg_gph.mean(dim = "Time") - july_gph.mean(dim = "Time")